In [51]:
# !pip install earthengine-api

In [52]:
import ee

# Google Earth Engine API 인증 (최초 1회 실행)
ee.Authenticate()

True

In [53]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [54]:
# data = pd.read_csv("./make_file/(보정)_서울열선_광진도로.csv")
data = pd.read_csv("./make_file/(최종)_서울열선_광진도로.csv")

In [55]:
# Google Earth Engine 초기화
ee.Initialize()

In [56]:
SET_START_YEAR = 2019 # 2019인 경우, 2019년 12월 1일부터임.
SET_END_YEAR = 2025 # 2025인 경우, 2025년 2월 28일 까지임.

In [57]:
def get_temperature(lon, lat, year) :
    point = ee.Geometry.Point([lon, lat])
    
    # 특정 겨울 기간 (2023년 12월 ~ 2024년 2월)
    start_date = f'{year}-12-01'
    end_date = f'{year+1}-02-28'
    
    # Landsat 8 데이터 가져오기
    landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .select('ST_B10')  # 지표면 온도 밴드
    
    # 🛠️ 올바른 변환 적용
    def convert_lst(image):
        lst_radiance = image.select('ST_B10').multiply(0.00341802).add(149.0)  # 방사밝기 변환
        lst_kelvin = lst_radiance  # 이미 켈빈 단위
        lst_celsius = lst_kelvin.subtract(273.15)  # 켈빈 → 섭씨 변환
        return lst_celsius.rename('surface_temperature')
    
    # 변환 적용
    lst_collection = landsat.map(convert_lst)
    
    # 🔍 평균 온도 계산
    mean_temp = lst_collection.mean()
    
    # 🔹 좌표에서 값을 추출
    temp_value = mean_temp.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30,
        bestEffort=True
    )
    
    avg_temp = temp_value.get('surface_temperature').getInfo()
    return avg_temp



In [58]:
data.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역', '고도의_차이', '경사각',
       '최근접_시설들_거리', '최근접_시설의_평균거리', '2019_평균_온도', '2020_평균_온도', '2021_평균_온도',
       '2022_평균_온도', '2023_평균_온도', '2024_평균_온도', '종합_평균_기온'],
      dtype='object')

In [59]:
for year in range(SET_START_YEAR, SET_END_YEAR, 1):
    column_name = f"{year}_평균_온도"
    
    if column_name in data.columns:
        print(f"{column_name} <- 이미 있습니다. ")
        continue  
    
    result = []
    for i in tqdm(range(len(data))): 
        lat = data.iloc[i]["중앙점_위도"]
        lon = data.iloc[i]["중앙점_경도"]
        temp = get_temperature(lon, lat, year)
        result.append(temp)

    data[column_name] = result
    print(len(result))


2019_평균_온도 <- 이미 있습니다. 
2020_평균_온도 <- 이미 있습니다. 
2021_평균_온도 <- 이미 있습니다. 
2022_평균_온도 <- 이미 있습니다. 
2023_평균_온도 <- 이미 있습니다. 
2024_평균_온도 <- 이미 있습니다. 


In [62]:
data.head(2)

,도로명,열선,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,도로_길이,...,경사각,최근접_시설들_거리,최근접_시설의_평균거리,2019_평균_온도,2020_평균_온도,2021_평균_온도,2022_평균_온도,2023_평균_온도,2024_평균_온도,종합_평균_기온
0,일원로,1,3차로,37.491015,127.080765,37.490648,127.080938,37.490832,127.080852,43.557286,...,0.298094,"165.17299640951381 ,141.88730282955694 ,165.97...",194.979965,3.083744,0.881741,-2.871131,-1.719942,-4.994405,-0.762408,-1.063733
1,자곡로5길,1,주거지 도로,37.472343,127.096176,37.472057,127.094614,37.472200,127.095395,218.130971,...,3.326804,"62.36827510450082 ,84.24249385836922 ,84.24249...",90.747029,2.331096,0.151180,-5.008077,-2.071998,-4.362071,-1.637150,-1.766170


In [61]:
avg_values_list = []

for i in tqdm(range(len(data))): 
    get_temp_colums = []
    
    temp_columns = data.columns
    for n in range(len(temp_columns)) :
        temp = temp_columns[n].replace("_", " ")
        if " 평균 온도" in temp :
            get_temp_colums.append(temp_columns[n])

    # print(get_temp_colums)
    temp_sum = 0
    for n in range(len(get_temp_colums)) :
        temp_sum += data.iloc[i][get_temp_colums[n]] 

    
    temp_avg = round(temp_sum / len(get_temp_colums), 6)
    avg_values_list.append(temp_avg)
        
data['종합_평균_기온'] = avg_values_list

100%|████████████████████████████████████| 7212/7212 [00:00<00:00, 13147.68it/s]


In [48]:
data.isnull().sum()

도로명              0
열선               0
도로 종류            0
시작점_위도           0
시작점_경도           0
종료점_위도           0
종료점_경도           0
중앙점_위도           0
중앙점_경도           0
도로_길이            0
행정동              0
도로폭             74
도로규모            74
행정구역             0
고도의_차이           0
경사각              0
최근접_시설들_거리       0
최근접_시설의_평균거리     0
2019_평균_온도       0
2020_평균_온도       0
2021_평균_온도       0
2022_평균_온도       0
2023_평균_온도       0
2024_평균_온도       0
종합_평균_기온         0
dtype: int64

In [65]:
data['종합_평균_기온'].head()

0   -1.063733
1   -1.766170
2   -0.948210
3   -0.569161
4   -2.914910
Name: 종합_평균_기온, dtype: float64

In [63]:
try :
    save_url = "./make_file/(표면)_서울열선_광진도로.csv"
    data.to_csv(save_url, encoding="UTF-8", index=False)
    print(f"{save_url} 저장 완료.")
except OSError as e :
    print(e)

./make_file/(최종)_서울열선_광진도로.csv 저장 완료.
